# Goal here is to correct the label before the finetuning of ESM2
***
## I. Generate the DF_info and the DF_embeddings 
## II. Tryout multiple algoritms : 
> A. Ensemble method : Random Forest <br>
> B. Ensemble method : XGboost (?) <br>
> C. NN : Feed-forward Neural Network (FNN) <br>
> D. NN : Recurrent Neural Networks ; LSTM <br>
> E. NN : Graph Neural Networks  <br>
***

### I.

In [1]:
import torch
from torch import nn 
from torch.utils.data import Dataset , DataLoader
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder , label_binarize , OneHotEncoder
from sklearn.metrics import average_precision_score
import os 
import pandas as pd
import numpy as np

from collections import Counter
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 


/home/concha-eloko/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


> Open the Dataframe

In [2]:
path_work = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023"
path_work = "/media/concha-eloko/Linux/PPT_clean"

# Open the DF
DF_info = pd.read_csv(f"{path_work}/DF_Dpo.final.2705.tsv", sep = "\t" ,  header = 0 )
# Open the embeddings
DF_embeddings = pd.read_csv(f"{path_work}/Dpo.2705.embeddings.ultimate.csv", sep = ",", header= None )
DF_embeddings.rename(columns={0: 'index'}, inplace=True)

# Filter the DF :
DF_info_filtered = DF_info[~DF_info["KL_type_LCA"].str.contains("\\|")]
DF_info_ToReLabel = DF_info[DF_info["KL_type_LCA"].str.contains("\\|")]
all_data = pd.merge(DF_info_filtered , DF_embeddings , on = "index")

> Label encoder ; define test_set

In [3]:
all_data = all_data.drop_duplicates(subset = ["Infected_ancestor","index"] , keep = "first").reset_index(drop=True)
dico_infected_a = dict(Counter(all_data["Infected_ancestor"]))
dico_phage = dict(Counter(all_data["Phage"]))

# Create ew categories for the punctual phages and ancestors : 
all_data["labeled_ancestor"] = all_data["Infected_ancestor"].apply(lambda x : x if dico_infected_a[x] > 1 else "punctual_ancestor")
all_data["labeled_phage"] = all_data["Phage"].apply(lambda x : x if dico_phage[x] > 1 else "punctual_phage")

# Label encode the categories and the labels : 
LE  = LabelEncoder()
OHE = OneHotEncoder()
all_data["KL_type_LCA"] = LE.fit_transform(all_data["KL_type_LCA"])

# all_data["labeled_ancestor"] = LE.fit_transform(all_data["labeled_ancestor"])
# all_data["labeled_phage"] = LE.fit_transform(all_data["labeled_phage"])

In [4]:
all_data

,Phage,KL_type_LCA,Infected_ancestor,Protein_name,Dataset,index,seq,prophage_id,1,2,...,1273,1274,1275,1276,1277,1278,1279,1280,labeled_ancestor,labeled_phage
0,GCF_016651625.1__phage29,95,GCF_016651625.1,GCF_016651625.1__phage29__142,ppt,ppt__1,MSVPNQTPYNIYTANGLTTVFTYEFYIISASDLRVSINGDVVTSGY...,prophage_12186,-0.018416,0.022387,...,0.087907,0.015800,0.025778,0.065790,0.034045,-0.070899,0.016068,0.065339,GCF_016651625.1,GCF_016651625.1__phage29
1,GCF_016651625.1__phage29,95,GCF_016651625.1,GCF_016651625.1__phage29__150,anubis,anubis__0,MRANLIKTNFTAGEISPRLMGRVDIARYANGAKIIENAVCVVQGGV...,prophage_12186,0.036016,0.005938,...,0.008772,0.010556,-0.049738,-0.012549,0.092624,-0.136602,-0.191378,0.135658,GCF_016651625.1,GCF_016651625.1__phage29
2,GCF_016651625.1__phage12,95,GCF_016651625.1,GCF_016651625.1__phage12__59,ppt,ppt__4,MSISKRNFLKAVSCAYFFYSFKALTKVNQPIEDYISTKDKNTWPSK...,prophage_924,0.026004,0.024372,...,0.018206,0.036751,-0.032549,0.064112,0.061520,-0.024423,-0.027998,0.028089,GCF_016651625.1,punctual_phage
3,GCF_019928025.1__phage0,95,n1471,GCF_019928025.1__phage0__10,ppt,ppt__4,MSISKRNFLKAVSCAYFFYSFKALTKVNQPIEDYISTKDKNTWPSK...,prophage_2929,0.026004,0.024372,...,0.018206,0.036751,-0.032549,0.064112,0.061520,-0.024423,-0.027998,0.028089,punctual_ancestor,punctual_phage
4,GCF_004313505.1__phage4,39,GCF_004313505.1,GCF_004313505.1__phage4__113,anubis,anubis__5,MSEYDTGNPVPSASMPDAWDNMQSIDKFVNSSDETITTRTGQQLDT...,prophage_11091,-0.023648,0.052674,...,0.068538,-0.051192,0.026481,0.069100,0.017813,-0.103797,0.018961,0.117058,GCF_004313505.1,punctual_phage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9357,GCF_015209405.1__phage17,70,81.9/1004099,GCF_015209405.1__phage17__0,anubis,anubis__1643,MASIKELPRWEDEVYQIARGDKVEGGVGGIANMQAKTLAERTRYLK...,prophage_12,-0.014377,0.119705,...,0.108840,0.008335,0.026747,0.046616,0.033648,-0.092224,0.048720,0.034757,81.9/1004099,punctual_phage
9358,GCF_004311345.1__phage11,85,GCF_004311345.1,GCF_004311345.1__phage11__99,anubis,anubis__1644,MTANYPASILPPNATAVERAIDRASAAALERLPVYLIRWVKDPDSC...,prophage_11944,-0.001691,-0.067717,...,0.057788,0.004341,0.007389,-0.081588,0.100995,-0.045545,0.021685,0.013167,GCF_004311345.1,punctual_phage
9359,GCF_900172635.1__phage3,26,GCF_900172635.1,GCF_900172635.1__phage3__1811,anubis,anubis__1647,MTANYPASILPPNATAVERAIDRASAAALERLPVYLIRWVKDPDSC...,prophage_11522,-0.002816,-0.055336,...,0.059700,0.004641,0.016676,-0.071711,0.095491,-0.055784,0.015615,0.000201,punctual_ancestor,punctual_phage
9360,GCF_013604525.1__phage18,82,n49324932,GCF_013604525.1__phage18__52,anubis,anubis__1649,MANIEKLGSSSPEVLLKNATNLDKLVNGRESESLPDRFGVLRKTWH...,prophage_2576,-0.000031,0.046085,...,0.077754,-0.024675,0.026408,0.016113,0.043904,-0.091418,0.078778,0.105842,punctual_ancestor,punctual_phage


In [5]:
all_data[all_data["index"] == "anubis__63"]

,Phage,KL_type_LCA,Infected_ancestor,Protein_name,Dataset,index,seq,prophage_id,1,2,...,1273,1274,1275,1276,1277,1278,1279,1280,labeled_ancestor,labeled_phage
3687,GCF_900513655.1__phage15,31,n52565256,GCF_900513655.1__phage15__689,anubis,anubis__63,MAFNPELGSTSPAVLLDNAERLDKLVNGSALTEPDRAGVNLDTWRG...,prophage_43,-0.012539,0.044253,...,0.068,0.029141,0.011152,0.04488,0.087621,-0.075892,-0.08788,0.109675,n52565256,GCF_900513655.1__phage15
3688,GCF_900608435.1__phage24,8,n29342934,GCF_900608435.1__phage24__1684,anubis,anubis__63,MAFNPELGSTSPAVLLDNAERLDKLVNGSALTEPDRAGVNLDTWRG...,prophage_43,-0.012539,0.044253,...,0.068,0.029141,0.011152,0.04488,0.087621,-0.075892,-0.08788,0.109675,n29342934,GCF_900608435.1__phage24
3689,GCF_902157075.1__phage12,7,n3272,GCF_902157075.1__phage12__2941,anubis,anubis__63,MAFNPELGSTSPAVLLDNAERLDKLVNGSALTEPDRAGVNLDTWRG...,prophage_43,-0.012539,0.044253,...,0.068,0.029141,0.011152,0.04488,0.087621,-0.075892,-0.08788,0.109675,n3272,GCF_902157075.1__phage12
3690,GCF_002243445.1__phage1,7,79.4/1003357,GCF_002243445.1__phage1__352,anubis,anubis__63,MAFNPELGSTSPAVLLDNAERLDKLVNGSALTEPDRAGVNLDTWRG...,prophage_5946,-0.012539,0.044253,...,0.068,0.029141,0.011152,0.04488,0.087621,-0.075892,-0.08788,0.109675,79.4/1003357,GCF_002243445.1__phage1
3691,GCF_002812385.1__phage14,7,n3352,GCF_002812385.1__phage14__2991,anubis,anubis__63,MAFNPELGSTSPAVLLDNAERLDKLVNGSALTEPDRAGVNLDTWRG...,prophage_43,-0.012539,0.044253,...,0.068,0.029141,0.011152,0.04488,0.087621,-0.075892,-0.08788,0.109675,n3352,GCF_002812385.1__phage14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3769,GCF_900514005.1__phage24,8,GCF_900514005.1,GCF_900514005.1__phage24__9,anubis,anubis__63,MAFNPELGSTSPAVLLDNAERLDKLVNGSALTEPDRAGVNLDTWRG...,prophage_12979,-0.012539,0.044253,...,0.068,0.029141,0.011152,0.04488,0.087621,-0.075892,-0.08788,0.109675,punctual_ancestor,punctual_phage
3770,GCF_016939675.1__phage2,8,n29952995,GCF_016939675.1__phage2__1945,anubis,anubis__63,MAFNPELGSTSPAVLLDNAERLDKLVNGSALTEPDRAGVNLDTWRG...,prophage_1511,-0.012539,0.044253,...,0.068,0.029141,0.011152,0.04488,0.087621,-0.075892,-0.08788,0.109675,n29952995,GCF_016939675.1__phage2
3771,GCF_003860995.1__phage1,8,87.2/1002978,GCF_003860995.1__phage1__124,anubis,anubis__63,MAFNPELGSTSPAVLLDNAERLDKLVNGSALTEPDRAGVNLDTWRG...,prophage_35,-0.012539,0.044253,...,0.068,0.029141,0.011152,0.04488,0.087621,-0.075892,-0.08788,0.109675,87.2/1002978,punctual_phage
3772,GCF_008123385.1__phage25,69,n4495,GCF_008123385.1__phage25__1,anubis,anubis__63,MAFNPELGSTSPAVLLDNAERLDKLVNGSALTEPDRAGVNLDTWRG...,prophage_7001,-0.012539,0.044253,...,0.068,0.029141,0.011152,0.04488,0.087621,-0.075892,-0.08788,0.109675,n4495,GCF_008123385.1__phage25


> Encode the Infected_ancestor and the Phage

In [6]:
import pprint
pp = pprint.PrettyPrinter(width = 200, compact = True)
pp.pprint(dico_infected_a)

{'12.8/97': 1,
 '20.6/100': 1,
 '27/100': 1,
 '34.2/100': 15,
 '34.4/97': 2,
 '39.3/100': 3,
 '43.3/100': 2,
 '43.6/100': 1,
 '46.2/100': 1,
 '66/100': 2,
 '68.5/100': 1,
 '68.5/1003403': 2,
 '69.4/100': 1,
 '70.4/1003874': 7,
 '70.7/100': 1,
 '71.1/100': 1,
 '71.2/100': 1,
 '71.3/1002083': 2,
 '72.1/100': 2,
 '72.2/100': 1,
 '72.2/1001586': 2,
 '72.5/1001797': 1,
 '72.7/1004661': 2,
 '72/1002657': 3,
 '73.1/1004872': 2,
 '73.2/1002871': 5,
 '73.2/1005650': 1,
 '73.3/1004884': 2,
 '73.4/1004125': 1,
 '73.4/1005645': 4,
 '73.6/100100': 1,
 '73.6/1001413': 1,
 '73.6/1004940': 2,
 '73.7/100': 1,
 '73.7/1001407': 2,
 '73.7/1002034': 2,
 '73.7/1002116': 1,
 '74.3/100144': 2,
 '74.3/1001743': 1,
 '74.3/1002378': 7,
 '74.5/1001465': 1,
 '74.5/1002052': 1,
 '74.6/1001169': 1,
 '74.6/1001616': 2,
 '74.6/1003860': 1,
 '74.6/1004875': 1,
 '74.7/100': 1,
 '74.7/1001691': 2,
 '74.7/100743': 1,
 '74.8/100': 1,
 '74.8/1001649': 2,
 '74.8/1001923': 1,
 '74.8/1003310': 3,
 '74.8/1003562': 2,
 '74.8/100

> Include the weigth class

In [5]:
class_samples = np.bincount(all_data.KL_type_LCA)
class_weights = 1. / torch.tensor(np.sqrt(class_samples), dtype=torch.float)
# define loss function
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)
class_samples

array([ 100,   97,    1,  247,   12,    4,  130,  542, 1241,   27,   13,
          6,   51,   91,   68,    7,   24,    1,   23,   17,   26,    7,
         50,    1,   34,   36,   12,   21,   20,   47,   34,   82,    1,
          3,    3,    3,   26,    6,   13,  131,   15,    3,   13,    9,
          1,   42,    7,    4,    1,   55,  132,    7,   41,    9,   18,
          1,   17,    8,    7,   13,   22,    2,    5,    7,    7,   24,
        597,    3,   26,   75,  347,   30,   72,   90,  128,  304,  352,
         14,  117,  105,   38,  148,   90,   15,    1,   16,   21,   99,
          5,   88,   26,   15,    1,   14,    9,   54,   75,   80,  691,
         25,    3,   23,  107,   45,   29,   12,   23,   16,   60,    6,
          1,    9,   72,    5,  142,   65,  968,   10,   13,   39,   30,
         12,   88,   11,   46,    9,   19])

In [14]:
class_samples[96]

75

> DataLoader 

In [ ]:
all_data["KL_type_LCA"] = LE.fit_transform(all_data["KL_type_LCA"])
all_data["labeled_ancestor"] = LE.fit_transform(all_data["labeled_ancestor"])
all_data["labeled_phage"] = LE.fit_transform(all_data["labeled_phage"])


In [42]:
#all_data = pd.get_dummies(all_data, columns=["labeled_ancestor", "labeled_phage"])
all_data
labeled_ancestor = OHE.fit_transform(all_data[["labeled_ancestor"]]).toarray()
labeled_ancestor

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [7]:
class MultiDomainDataset(Dataset):
    def __init__(self, all_data):   
        self.embeddings = torch.tensor(all_data[[int(i) for i in range(1, 1281)]].values, dtype=torch.float)
        labeled_ancestor = OHE.fit_transform(all_data[["labeled_ancestor"]]).toarray()
        self.ancestor = torch.from_numpy(labeled_ancestor).to(torch.float)
        labeled_phage = OHE.fit_transform(all_data[["labeled_phage"]]).toarray()
        self.prophage_instance = torch.from_numpy(labeled_phage).to(torch.float)
        self.labels = torch.tensor(all_data["KL_type_LCA"].values, dtype=torch.long) 

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item_domain1 = self.embeddings[idx]
        item_domain2 = self.ancestor[idx]
        item_domain3 = self.prophage_instance[idx]
        item_domain4 = self.labels[idx]
        return item_domain1, item_domain2, item_domain3 , item_domain4 

In [8]:
X_temp, X_eval, y_temp, y_eval = train_test_split(all_data, all_data["KL_type_LCA"], test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

train_singledata = MultiDomainDataset(X_train)
test_singledata = MultiDomainDataset(X_test)

train_single_loader = DataLoader(train_singledata, batch_size=12, shuffle=True, num_workers=4)
test_single_loader = DataLoader(test_singledata, batch_size=12, shuffle=True, num_workers=4)

In [9]:
train_singledata[0]

(tensor([-0.0014, -0.0642,  0.0497,  ..., -0.0484,  0.0262,  0.0083]),
 tensor([0., 0., 0.,  ..., 0., 0., 0.]),
 tensor([0., 0., 0.,  ..., 0., 0., 1.]),
 tensor(73))

***
### II. 

> The model

In [ ]:
class SingleDomainModel_3L(nn.Module):
    def __init__(self):
        super(SingleDomainModel_4L, self).__init__()
        self.lstm = nn.LSTM(input_size=1280, hidden_size=512, num_layers=2, batch_first=True, bidirectional=True)
        
        self.layer1 = nn.Linear(in_features=512*2, out_features=1024)  # Increased out_features
        self.layer2 = nn.Linear(in_features=1024, out_features=512)
        self.layer3 = nn.Linear(in_features=512, out_features=127)  # Increase nodes in next layer
        
    def forward(self, domain1_data):
        out, _ = self.lstm(domain1_data.unsqueeze(1))  # Add extra dimension for sequence length
        out = F.relu(out.squeeze(1))
        out = F.relu(self.layer1(out))  # Remove the sequence length dimension
        out = F.relu(self.layer2(out))  # Add ReLU activation function
        out = self.layer3(out)  # Output layer
        return out

In [17]:
model = SingleDomainModel()
print(model)

SingleDomainModel(
  (lstm): LSTM(1280, 512, num_layers=2, batch_first=True, bidirectional=True)
  (layer1): Linear(in_features=1024, out_features=1024, bias=True)
  (layer2): Linear(in_features=1024, out_features=512, bias=True)
  (layer3): Linear(in_features=512, out_features=256, bias=True)
  (layer4): Linear(in_features=256, out_features=128, bias=True)
  (layer5): Linear(in_features=128, out_features=127, bias=True)
)


In [139]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Use GPU if available

model = SingleDomainModel_4L()
model = model.to(device)  # Move model to GPU

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Increase initial learning rate
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)  # Decrease learning rate every 30 epochs by 0.1

history = {
    "train_loss": [],
    "train_accuracy": [],
    "val_loss": [],
    "val_accuracy": [],
    "val_auprc": {},  # Store AUPRC for each class
    "val_auroc": {},  # Store AUROC for each class
    "confusion_matrix": {},  # Store confusion matrix for each class
}


for epoch in range(100):  # Increase number of epochs
    # Switch model to training mode
    model.train()
    epoch_correct = 0
    epoch_all = 0
    epoch_loss = 0.0
    for i, (embeddings , labels) in enumerate(train_single_loader):
        embeddings = embeddings.to(device)  # Move data to GPU
        labels = labels.to(device)  # Move labels to GPU

        outputs = model(embeddings)
        loss = criterion(outputs, labels)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Compute accuracy
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        epoch_correct += correct
        epoch_all += labels.size(0)        
        # Accumulate loss
        epoch_loss += loss.item()
    # Compute metrics for this epoch
    epoch_loss = epoch_loss / len(train_single_loader)
    epoch_accuracy = epoch_correct / epoch_all
    
    # Step the scheduler
    scheduler.step()

    # Switch model to evaluation mode
with torch.no_grad():
    val_correct = 0
    val_all = 0
    val_loss = 0.0
    val_predictions = []
    val_true_labels = []
    for embeddings, labels in test_single_loader:
        embeddings = embeddings.to(device)  # Move data to GPU
        labels = labels.to(device)  # Move labels to GPU

        outputs = model(embeddings)
        loss = criterion(outputs, labels)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        val_correct += correct
        val_all += labels.size(0)
        val_loss += loss.item()
        # Save predictions and true labels for AUPRC and AUROC calculation
        val_predictions.append(outputs.cpu().numpy())
        val_true_labels.append(labels.cpu().numpy())
    val_loss = val_loss / len(test_single_loader)
    val_accuracy = val_correct / val_all
    # Calculate AUPRC and AUROC for each class
    val_predictions = np.concatenate(val_predictions, axis=0)
    val_true_labels = np.concatenate(val_true_labels, axis=0)
    for class_index in range(127):  # Update this with your number of classes
        # AUROC and AUPRC calculation
        try:
            roc_auc = roc_auc_score(val_true_labels == class_index, val_predictions[:, class_index])
            history["val_auroc"][class_index] = roc_auc
            class_auprc = average_precision_score(val_true_labels == class_index, val_predictions[:, class_index])
            history["val_auprc"][class_index] = class_auprc
        except:
            history["val_auroc"][class_index] = np.nan
            history["val_auprc"][class_index] = np.nan
        # Confusion matrix calculation
        matrix = confusion_matrix(val_true_labels, val_predictions[:, class_index].round())
        history["confusion_matrix"][class_index] = matrix.tolist()  # Convert numpy array to list for JSON serializability

    
# Save model after training
torch.save(model.state_dict(), f"{path_work}/train_nn/SingleDomain.LSTM.model")


KeyboardInterrupt: 

In [ ]:
import json
with open(f"{path_work}/train_nn/SingleDomain.LSTM.out" , "w") as outfile :
    outfile.write(json.dumps(history))

In [ ]:
#!/bin/bash
#BATCH --job-name=NN_trials__
#SBATCH --qos=short
#SBATCH --ntasks=1 
#SBATCH --cpus-per-task=10 
#SBATCH --mem=50gb 
#SBATCH --time=1-00:00:00 
#SBATCH --output=NN_trials__%j.log 

source /storage/apps/ANACONDA/anaconda3/etc/profile.d/conda.sh
conda activate embeddings

python /home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/train_nn/script_files/train_single_domain.py